In [1]:
import pandas as pd 
from dotenv import dotenv_values
from sqlalchemy import create_engine
import sqlalchemy as db

In [3]:
import matplotlib_inline

ModuleNotFoundError: No module named 'matplotlib'

In [2]:
# read data 
df_order = pd.read_csv('/Users/newuser/ali/project/kampus merdeka/repo_dibimbing/dataset/Data latihan sql/Sample - Store - Orders.csv')
df_people = pd.read_csv('/Users/newuser/ali/project/kampus merdeka/repo_dibimbing/dataset/Data latihan sql/Sample - Store - People.csv')
df_returns = pd.read_csv('/Users/newuser/ali/project/kampus merdeka/repo_dibimbing/dataset/Data latihan sql/Sample - Store - Returns.csv')

In [16]:
df_order['order_date']= pd.to_datetime(df_order['order_date'])
df_order['ship_date']= pd.to_datetime(df_order['ship_date'])


In [4]:
#create lower all coloumn
df_order.columns = map(str.lower, df_order.columns)
df_people.columns = map(str.lower, df_people.columns)
df_returns.columns = map(str.lower, df_returns.columns)
#replace char space to undnerscore
df_order.columns = df_order.columns.str.replace(' ', '_')
df_people.columns = df_people.columns.str.replace(' ', '_')
df_returns.columns = df_returns.columns.str.replace(' ', '_')

In [17]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   order_id       9994 non-null   object        
 1   order_date     9994 non-null   datetime64[ns]
 2   ship_date      9994 non-null   datetime64[ns]
 3   ship_mode      9994 non-null   object        
 4   customer_id    9994 non-null   object        
 5   customer_name  9994 non-null   object        
 6   segment        9994 non-null   object        
 7   country        9994 non-null   object        
 8   city           9994 non-null   object        
 9   state          9994 non-null   object        
 10  postal_code    9994 non-null   int64         
 11  region         9994 non-null   object        
 12  product_id     9994 non-null   object        
 13  category       9994 non-null   object        
 14  sub-category   9994 non-null   object        
 15  product_name   9994 n

In [9]:
#connect ke database
config = dotenv_values(".env")
# print(config)
def con_db():
    url = "{d}://{u}:{p}@{h}:{port}/{db}".\
            format(d="postgresql",
            u=config['POSTGRES_USER'],
            p=config['POSTGRES_PASSWORD'],
            h=config['POSTGRES_HOST'],
            port=config['POSTGRES_PORT'],
            db=config['POSTGRES_DB'])
    engine = db.create_engine(url)
    return engine

In [10]:
def insert_csv_order():
    db = con_db().connect()
    df_order.to_sql(
        'superstore_order', 
        db,
        schema='dibimbing_bi', 
        if_exists='replace',
        index=False,
        index_label='id',
        method='multi'
        )
    db.close()
    return{"result":"procces insert data order done"}
def insert_csv_people():
    db = con_db().connect()
    df_people.to_sql(
        'superstore_people', 
        db,
        schema='dibimbing_bi', 
        if_exists='replace',
        index=False,
        index_label='id',
        method='multi'
        )
    db.close()
    return{"result":"procces insert data people done"}
def insert_csv_return():
    db = con_db().connect()
    df_returns.to_sql(
        'superstore_returns', 
        db,
        schema='dibimbing_bi', 
        if_exists='replace',
        index=False,
        index_label='id',
        method='multi'
        )
    db.close()
    return{"result":"procces insert data returns done"}

insert_csv_order()
# insert_csv_people()
# insert_csv_return()

{'result': 'procces insert data returns done'}

In [18]:
insert_csv_order()

{'result': 'procces insert data order done'}

In [ ]:
"""
select 
distinct (so.order_id) ,
so.ship_mode ,
so.customer_name 
from superstore_order so 
left join superstore_returns on 
order by so.order_id  asc 
limit 30
offset 10

"""

"""
select 
count(order_id) as count_order

from superstore."Sample - Superstore" ss 
right join dibimbing_bi.superstore_returns sr on ss."Order ID" = sr.order_id 
"""

In [ ]:
""" 
--total data return 3 bulan terkahir tahun 2014

with contoh_cte as (
select
	so.order_id as order_id,
	so.customer_name,
	so.product_name,
	sr.order_id as return_order_id,
	so.order_date as return_id,
	so.product_id,
	date_part('year', so.order_date::date) as year_order,
	so.order_date 
from
	superstore_order so
join superstore_returns sr on
	sr.order_id = so.order_id
where 
--	sr.order_id in 
date_part('year', so.order_date::date) = 2014
),
sub_cte as (select
	count(*) as total_return_order,
	to_char(order_date , 'YYYY-MM') as year_month
from
	contoh_cte
--where date_part('year', so.order_date::date)
where to_char(order_date , 'MM') in ('10','11','12')
group by
order_date
)
select 
sum(total_return_order),
year_month
from sub_cte
group by 
sub_cte.year_month



--5 most product best sell in 2015
select  
order_id,
order_date
from 
superstore_order so 
where 
total order_date th 2015
limit 
5
order by 
order_date 


--5 most product best sell in 2015
select  
count(product_id),
product_name 
from 
superstore_order so 
where 
order_date  = 2015
limit 
5
order by 
count(product_id) asc 

-- 
select 
product_name,
count(quantity)
from
superstore_order so 
where 
date_part('year', so.order_date::date) = 2015
group by 
product_name 
order by 
count(quantity) desc 
limit 5;

-- total data tahun ke tahun

select
	count(order_id) ,
	to_char(order_date , 'YYYY') as tahun,
	sum(profit) as total_profit
	--(select date_part('year', so.order_date::date) as tahun
from
	superstore_order so 
group by
	tahun



select
	date_part('year', o.order_date :: date) as tahun ,
	count(order_id),
	sum(profit)
from superstore_order o 
group by
tahun

select count(order_id)
from 
superstore_order so 
where date_part('year', order_date :: date) = 2014

"""